In [1]:
# !mkdir /root/.ssh
# !cp /content/drive/MyDrive/deploy_keys/id_ed25519* /root/.ssh/
# !ssh-keyscan -t ed25519 github.com >> ~/.ssh/known_hosts
# !ssh -T git@github.com

In [2]:
!git clone https://github.com/ddudakov/DatasetEditing.git

Cloning into 'DatasetEditing'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (500/500), done.
remote: Compressing objects: 100% (435/435), done.
remote: Total 500 (delta 53), reused 489 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (500/500), 17.07 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (53/53), done.


***Установка пакетов***

In [4]:
cd DatasetEditing

[Errno 2] No such file or directory: 'DatasetEditing'
/content/DatasetEditing


In [5]:
#!pip install git+ssh://git@github.com/ddudakov/DatasetEditing.git

In [6]:
import os, sys

sys.path.append(os.path.join(os.getcwd(), "SAM/segment_anything"))
sys.path.append(os.path.join(os.getcwd(), "GroundingDINO"))

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', '/content/DatasetEditing/SAM/segment_anything', '/content/DatasetEditing/GroundingDINO']


In [8]:
!python -m pip install -e GroundingDINO

Obtaining file:///content/DatasetEditing/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.3/833.3 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.0 

In [9]:
import argparse
import os
import copy

import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
from torchvision.ops import box_convert

# Grounding DINO
from GroundingDINO.groundingdino.datasets import transforms as T
from GroundingDINO.groundingdino.models import build_model
from GroundingDINO.groundingdino.util import box_ops
from GroundingDINO.groundingdino.util.slconfig import SLConfig
from GroundingDINO.groundingdino.util.utils import clean_state_dict, get_phrases_from_posmap
from GroundingDINO.groundingdino.util.inference import annotate, load_image, predict

import supervision as sv
# segment anything
from SAM.segment_anything import build_sam, SamPredictor
import cv2
import numpy as np
import matplotlib.pyplot as plt


import PIL
import requests
import torch
from io import BytesIO



from huggingface_hub import hf_hub_download

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


**Загружаем Grounding DINO**

In [10]:
def load_model_hf(repo_id, filename, ckpt_config_filename, device='cpu'):
    cache_config_file = hf_hub_download(repo_id=repo_id, filename=ckpt_config_filename)

    args = SLConfig.fromfile(cache_config_file)
    model = build_model(args)
    args.device = device

    cache_file = hf_hub_download(repo_id=repo_id, filename=filename)
    checkpoint = torch.load(cache_file, map_location='cpu')
    log = model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
    print("Model loaded from {} \n => {}".format(cache_file, log))
    _ = model.eval()
    return model

In [11]:
ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filename = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"

groundingdino_model = load_model_hf(ckpt_repo_id, ckpt_filename, ckpt_config_filename)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


GroundingDINO_SwinB.cfg.py:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

groundingdino_swinb_cogcoor.pth:   0%|          | 0.00/938M [00:00<?, ?B/s]

Model loaded from /root/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


**Загружаем SAM**

In [12]:
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

--2024-10-23 12:24:19--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.172.185.63, 18.172.185.104, 18.172.185.43, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.172.185.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   217MB/s    in 20s     

2024-10-23 12:24:39 (125 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [13]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

sam_checkpoint = 'sam_vit_h_4b8939.pth'
sam = build_sam(checkpoint=sam_checkpoint)
sam.to(device=DEVICE)
sam_predictor = SamPredictor(sam)

**Загружаем checkpoints для Background Editing и checkpoints для inpaint-anything**

In [14]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')
# https://drive.google.com/drive/folders/1hVh-eLY_J5TEryOtkofMpjgZ6ANu9r09?usp=sharing
# background editing
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/ckpt_editing/checkpoints.zip", 'r')
zip_ref.extractall("/content/DatasetEditing/editing_diffusion/")
zip_ref.close()

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/ckpt_editing/checkpoints1.zip", 'r')
zip_ref.extractall("/content/DatasetEditing/editing_diffusion/")
zip_ref.close()

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/ckpt_editing/ViT-B-16.zip", 'r')
zip_ref.extractall("/root/.cache/clip/")
zip_ref.close()

# inpaint anything
!cp -r '/content/drive/MyDrive/Colab Notebooks/pretrained_models' '/content/DatasetEditing/Inpaint-Anything/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**Загрузим датасет для экспериментов**

In [15]:
!rm -rf /content/sample_data && mkdir /content/sample_data && mkdir /content/sample_data/test

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/test.zip", 'r')
zip_ref.extractall("/content/sample_data/test")
zip_ref.close()

#TEXT_PROMPT = "bird"
BOX_TRESHOLD = 0.3
TEXT_TRESHOLD = 0.25

In [16]:
!rm -rf /content/sample_data/test_new
!mkdir /content/sample_data/test_new
# !cp -r /content/sample_data/test/EMU /content/sample_data/test_new
# !cp -r /content/sample_data/test/GRANDALA /content/sample_data/test_new
# !cp -r /content/sample_data/test/BOBOLINK /content/sample_data/test_new
# !cp -r /content/sample_data/test/SORA /content/sample_data/test_new
!cp -r /content/sample_data/test/ANIANIAU /content/sample_data/test_new
!cp -r /content/sample_data/test/CASSOWARY /content/sample_data/test_new

In [17]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [18]:
# TESTING
import random
import os
from pathlib import Path
import subprocess
TEXT_PROMPT = "bird"

dir_img_path = "/content/sample_data/test_new"
names = sorted([name for name in os.listdir(dir_img_path)])
scales = [0.05, 0.08, 0.11]
for name, scale in zip(names, scales):
  class_path = f"{dir_img_path}/{name}"
  TEXT_PROMPT = name
  prelim = f"{class_path}/preliminaries"
  os.makedirs(prelim, exist_ok=True)
  for img in os.listdir(class_path):
    filepath = os.path.join(class_path, img)
    if os.path.isfile(filepath):
        img_path = f"{class_path}/{img}"
        image_source, image = load_image(img_path)
        boxes, logits, phrases = predict(
            model=groundingdino_model,
            image=image,
            caption=TEXT_PROMPT,
            box_threshold=BOX_TRESHOLD,
            text_threshold=TEXT_TRESHOLD,
            device=DEVICE
        )
        # set image
        sam_predictor.set_image(image_source)
        # box: normalized box xywh -> unnormalized xyxy
        H, W, _ = image_source.shape
        boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.Tensor([W, H, W, H])
        transformed_boxes = sam_predictor.transform.apply_boxes_torch(boxes_xyxy, image_source.shape[:2]).to(DEVICE)
        masks, _, _ = sam_predictor.predict_torch(
                    point_coords = None,
                    point_labels = None,
                    boxes = transformed_boxes,
                    multimask_output = False,
                    )
        mask_path = f"{prelim}/{Path(img).stem}_mask.png"
        image_mask = masks[0][0].cpu().numpy()
        image_mask_pil = Image.fromarray(image_mask) # our mask
        image_mask_pil.save(mask_path)
        !python /content/DatasetEditing/Inpaint-Anything/remove_anything.py --input_img {img_path} --input_mask {mask_path} --dilate_kernel_size 15 \
        --output_dir {prelim} --lama_config "/content/DatasetEditing/Inpaint-Anything/lama_configs/default.yaml" --lama_ckpt "/content/DatasetEditing/Inpaint-Anything/pretrained_models/big-lama"
        !python /content/DatasetEditing/resize_obj.py --img_path {img_path} --mask_path {mask_path} --output_path {prelim} --scale {scale}
        # load img_rescaled and mask_rescaled
        background_path = f"{prelim}/{Path(img).stem}_mask_background.png"
        img_rescaled_path = f"{prelim}/{Path(img).stem}_rescaled.png"
        mask_rescaled_path = f"{prelim}/{Path(img).stem}_mask_rescaled.png"
        !python /content/DatasetEditing/editing_diffusion/main.py -p "" --batch_size 1 -i {background_path} -i2 {img_rescaled_path}  \
        --mask {mask_rescaled_path} --output_path {class_path} --output_file {Path(img)} --skip_timesteps 0 --iterations_num 5 --invert_mask --classifier_scale 0. --y 0 --final_save_root {class_path} --background_complex 15 --hard --random_position --coarse_to_fine #--background_preservation_loss # --vid


Traceback (most recent call last):
  File "/content/DatasetEditing/Inpaint-Anything/remove_anything.py", line 5, in <module>
    from lama_inpaint import inpaint_img_with_lama
  File "/content/DatasetEditing/Inpaint-Anything/lama_inpaint.py", line 19, in <module>
    from saicinpainting.evaluation.utils import move_to_device
  File "/content/DatasetEditing/Inpaint-Anything/saicinpainting/evaluation/__init__.py", line 6, in <module>
    from saicinpainting.evaluation.losses.base_loss import SSIMScore, LPIPSScore, FIDScore
  File "/content/DatasetEditing/Inpaint-Anything/saicinpainting/evaluation/losses/base_loss.py", line 13, in <module>
    from models.ade20k import SegmentationModule, NUM_CLASS, segm_options
  File "/content/DatasetEditing/Inpaint-Anything/models/ade20k/__init__.py", line 1, in <module>
    from .base import *
  File "/content/DatasetEditing/Inpaint-Anything/models/ade20k/base.py", line 22, in <module>
    classes=pd.read_csv(classes_path),)
  File "/usr/local/lib/pyt

Traceback (most recent call last):
  File "/content/DatasetEditing/Inpaint-Anything/remove_anything.py", line 5, in <module>
    from lama_inpaint import inpaint_img_with_lama
  File "/content/DatasetEditing/Inpaint-Anything/lama_inpaint.py", line 19, in <module>
    from saicinpainting.evaluation.utils import move_to_device
  File "/content/DatasetEditing/Inpaint-Anything/saicinpainting/evaluation/__init__.py", line 6, in <module>
    from saicinpainting.evaluation.losses.base_loss import SSIMScore, LPIPSScore, FIDScore
  File "/content/DatasetEditing/Inpaint-Anything/saicinpainting/evaluation/losses/base_loss.py", line 13, in <module>
    from models.ade20k import SegmentationModule, NUM_CLASS, segm_options
  File "/content/DatasetEditing/Inpaint-Anything/models/ade20k/__init__.py", line 1, in <module>
    from .base import *
  File "/content/DatasetEditing/Inpaint-Anything/models/ade20k/base.py", line 22, in <module>
    classes=pd.read_csv(classes_path),)
  File "/usr/local/lib/pyt

KeyboardInterrupt: 

In [ ]:
print(DEVICE)